In [21]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import pickle
import seaborn as sns
import folium
import time
import re
import os
from datetime import datetime, date, timedelta
import warnings
warnings.filterwarnings('ignore')

In [26]:
options = Options()
options.set_preference('permissions.default.image', 0)
options.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', True)
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference('pdfjs.disabled', True)
#options.add_argument("--headless")
options.add_argument("--width=1080")
options.add_argument("--height=850")

In [23]:
distrital = gpd.read_file('anexos/distritos/distritos-peru@bogota-laburbano.geojson')
distrital = distrital[['nombdep','nombprov','nombdist','geo_point_2d']]
distrital = distrital[(distrital['nombdep']=='LIMA')|(distrital['nombdep']=='CALLAO')]
distrital = distrital.iloc[:-1,1:]
distrital['lat'] = None
distrital['lon'] = None

for row in range(distrital.shape[0]):
    distrital.iloc[row,3] = round(distrital.iloc[row,2]['lat'],2)
    distrital.iloc[row,4] = round(distrital.iloc[row,2]['lon'],2)

distrital = distrital[['nombprov','nombdist','lat','lon']]
distrital.columns = ['Provincia','Distrito','Lat','Lon']

pairs = {(row['Provincia'], row['Distrito']) for index, row in distrital.iterrows()}
pairs

{('BARRANCA', 'BARRANCA'),
 ('BARRANCA', 'PARAMONGA'),
 ('BARRANCA', 'PATIVILCA'),
 ('BARRANCA', 'SUPE'),
 ('BARRANCA', 'SUPE PUERTO'),
 ('CAJATAMBO', 'CAJATAMBO'),
 ('CAJATAMBO', 'COPA'),
 ('CAJATAMBO', 'GORGOR'),
 ('CAJATAMBO', 'HUANCAPON'),
 ('CAJATAMBO', 'MANAS'),
 ('CALLAO', 'BELLAVISTA'),
 ('CALLAO', 'CALLAO'),
 ('CALLAO', 'CARMEN DE LA LEGUA REYNOSO'),
 ('CALLAO', 'LA PERLA'),
 ('CALLAO', 'LA PUNTA'),
 ('CALLAO', 'MI PERU'),
 ('CALLAO', 'VENTANILLA'),
 ('CANTA', 'ARAHUAY'),
 ('CANTA', 'CANTA'),
 ('CANTA', 'HUAMANTANGA'),
 ('CANTA', 'HUAROS'),
 ('CANTA', 'LACHAQUI'),
 ('CANTA', 'SAN BUENAVENTURA'),
 ('CANTA', 'SANTA ROSA DE QUIVES'),
 ('CAÑETE', 'ASIA'),
 ('CAÑETE', 'CALANGO'),
 ('CAÑETE', 'CERRO AZUL'),
 ('CAÑETE', 'CHILCA'),
 ('CAÑETE', 'COAYLLO'),
 ('CAÑETE', 'IMPERIAL'),
 ('CAÑETE', 'LUNAHUANA'),
 ('CAÑETE', 'MALA'),
 ('CAÑETE', 'NUEVO IMPERIAL'),
 ('CAÑETE', 'PACARAN'),
 ('CAÑETE', 'QUILMANA'),
 ('CAÑETE', 'SAN ANTONIO'),
 ('CAÑETE', 'SAN LUIS'),
 ('CAÑETE', 'SAN VICENTE DE 

In [28]:
# Crear instancia del navegador
l = ['rain-3h','feel','temperature-2m','dew']
años = ['2020','2021','2022','2023','2024']
meses = ['enero','febrero','marzo','abril','mayo','junio','julio','agosto','septiembre','octubre','noviembre','diciembre']
base = 'https://www.ventusky.com/'

## SELECCIÓN POR INPUT (0 al 3 por posición en l)

pos = int(input('Ingrese posición de la lista l (0 al 3): '))
select = l[pos]

for row in range(distrital.shape[0]):
    caso = {'Año':[],
        'Provincia':[],
        'Distrito':[],
        'Mes':[],
        'Día':[],
        'Min':[],
        'Max':[]}
    
    prov = distrital.iloc[row,0]
    dist = distrital.iloc[row,1]
    lat = distrital.iloc[row,2]
    lon = distrital.iloc[row,3]

    # Crear enlace
    coords = f'?p={lat};{lon};10'
    access = f'&l={select}&m=icon'
    url = base + coords + access
        
    # Crear instancia del navegador
    driver = webdriver.Firefox(options=options)
    action = webdriver.ActionChains(driver)
    driver.implicitly_wait(300)
                
    # Abrir url en el driver
    driver.get(url)
    time.sleep(5)

    # Clickear dropdown
    fecha = driver.find_element(By.XPATH, '//a[@class="q j"]')
    fecha.click()
    
    for año in años:
        # Colocar año
        set_año = driver.find_element(By.XPATH, f'//select[@id="l"]/option[text()="{año}"]')
        set_año.click()
        time.sleep(2)
        
        for mes in meses:
            if año=='2024' and mes=='mayo': break
            else: pass
            # Colocar mes
            set_mes = driver.find_element(By.XPATH, f'//select[@id="h"]/option[text()="{mes}"]')
            set_mes.click()
            time.sleep(2)
                
            dias = driver.find_elements(By.XPATH, '//td[@class=" nk"]')
            ult_dia = int(dias[-1].text)

            rango = [1,15,ult_dia]
            
            for i in rango:
                dia = str(i).zfill(2)
                
                set_dia = driver.find_element(By.XPATH, f'//td/a[text()="{dia}"]')
                set_dia.click()
                time.sleep(2)
                
                try: action.move_by_offset(545,721).click().perform()
                except: action.move_by_offset(-191,332).click().perform()
                time.sleep(2)
                
                action.move_by_offset(191,-332).perform()
                min = driver.find_element(By.XPATH, '//span[@class="jd"]').text
                time.sleep(2)
                
                action.move_by_offset(191,332).click().perform()
                time.sleep(2)
                
                action.move_by_offset(-191,-332).perform()
                max = driver.find_element(By.XPATH, '//span[@class="jd"]').text
                time.sleep(2)

                caso['Año'] += [año]
                caso['Provincia'] += [prov]
                caso['Distrito'] += [dist]
                caso['Mes'] += [mes]
                caso['Día'] += [dia]
                caso['Min'] += [min]
                caso['Max'] += [max]

                print('------')
                print(f'{dia} de {mes}, {año}')
                print('Mínimo: ',min)
                print('Máximo: ',max,'\n') 
            
        # Exportar el caso específico
        caso_df = pd.DataFrame(caso)
        caso_df.to_csv(f'anexos/clima/{select}.csv',index=False,mode='a')
    
    # Cierra el navegador
    driver.close()    

Ingrese posición de la lista l (0 al 3):  2


------
01 de enero, 2020
Mínimo:  19 °C
Máximo:  18 °C 

------
15 de enero, 2020
Mínimo:  18 °C
Máximo:  19 °C 



KeyboardInterrupt: 

# Prueba

In [43]:
# Crear instancia del navegador
driver = webdriver.Firefox(options=options)
action = webdriver.ActionChains(driver)
driver.implicitly_wait(300)

# Abrir url en el driver
driver.get('https://www.ventusky.com/?p=-12.07;-77.0;10&l=feel&m=icon')
driver.maximize_window()
time.sleep(5)

# Clickear dropdown
fecha = driver.find_element(By.XPATH, '//a[@class="q j"]')
fecha.click()
time.sleep(2)

set_año = driver.find_element(By.XPATH, '//select[@id="l"]/option[text()="2020"]')
set_año.click()
time.sleep(2)

set_mes = driver.find_element(By.XPATH, '//select[@id="h"]/option[text()="enero"]')
set_mes.click()
time.sleep(2)

dias = driver.find_elements(By.XPATH, '//td[@class=" nk"]')
ult_dia = int(dias[-1].text)

for i in range(1,ult_dia+1):
    dia = str(i).zfill(2)

    set_dia = driver.find_element(By.XPATH, f'//td/a[text()="{dia}"]')
    set_dia.click()
    time.sleep(2)

    try: action.move_by_offset(545,721).click().perform()
    except: action.move_by_offset(-191,332).click().perform()
    time.sleep(2)

    action.move_by_offset(191,-332).perform()
    time.sleep(2)

    val1 = driver.find_element(By.XPATH, '//span[@class="jd"]').text
    print(val1)
    
    action.move_by_offset(191,332).click().perform()
    time.sleep(2)

    action.move_by_offset(-191,-332).perform()
    time.sleep(2)

    val2 = driver.find_element(By.XPATH, '//span[@class="jd"]').text
    print(val2)



23 °C
29 °C
23 °C
29 °C
21 °C
29 °C
23 °C
28 °C


NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:679:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:485:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.performActions@chrome://remote/content/marionette/driver.sys.mjs:1428:15
despatch@chrome://remote/content/marionette/server.sys.mjs:318:40
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:263:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:494:20
